# FISH-RT Probe Design Pipeline

**smfish-like-rt-probe-designer** - Design focused RT primers for SWIFT-seq using smFISH probe design principles (Oligostan).

## Pipeline Overview

```
Gene Symbols → [GTF/FASTA] → [Oligostan dG37] → [Filters] → [VCF SNP Analysis] → [BLAST] → Final Probes
```

1. **Step 1: Probe Design** - Fetch sequences, design probes, apply filters, analyze SNPs
2. **Step 2: Top Probe Selection** - Select top N probes per gene for BLAST validation
3. **Step 3: BLAST Validation** - Manual NCBI BLAST submission
4. **Step 4: BLAST Analysis** - Filter for unique genomic hits

---

## Setup: Install and Import Dependencies

In [ ]:
# Install dependencies if needed (uncomment to run)
# !pip install biopython pandas numpy pysam

In [ ]:
import os
import sys
import re
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, Markdown, HTML

# Add the repository to path
REPO_PATH = Path(os.getcwd())
if str(REPO_PATH) not in sys.path:
    sys.path.insert(0, str(REPO_PATH))

# Import pipeline modules
from config import FISH_RT_CONFIG, TEST_GENES_21, DG37_VALUES
from gene_fetcher import GeneSequenceFetcher
from utils.oligostan_core import design_fish_probes
from snp_analyzer import SNPCoverageAnalyzer
from output_generator import OutputGenerator

# Import CLI functions
from probe_picker_cli import select_top_probes, generate_fasta
from blast_analysis_cli import parse_blast_results, create_probe_identifier

# Biopython for FASTA
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

print("✅ All modules imported successfully")
print(f"📁 Repository path: {REPO_PATH}")

## Configuration Parameters

**Edit the parameters below to customize your analysis:**

In [ ]:
# ========================================
# CONFIGURATION - EDIT THESE PARAMETERS
# ========================================

# Gene list to process
GENE_LIST = [
    "Nanog",
    "Mecp2",
    "Xist",
]

# Or use the full test set (21 genes):
# GENE_LIST = TEST_GENES_21

# Output directory
OUTPUT_DIR = Path("/Users/gmgao/Dropbox/Caltech_PostDoc_GuttmanLab/constructs_and_smiFISH/smFISH_like_focusedRT-XCI/notebook_test")

# Input files (mm10)
GTF_PATH = "/Volumes/guttman/genomes/mm10/annotation/mm10.refGene.gtf.gz"
FASTA_PATH = "/Volumes/guttman/genomes/mm10/fasta/mm10.fa"
VCF_PATH = "/Volumes/guttman/genomes/mm10/variants/mgp.v5.merged.snps_all.dbSNP142.vcf.gz"

# VCF samples for allelic SNP detection
VCF_B6_SAMPLE = "C57BL_6NJ"
VCF_CAST_SAMPLE = "CAST_EiJ"

# Probe design parameters
PROBES_PER_GENE = 10  # Top N probes to select per gene
MIN_SNP_COVERAGE = 5  # Minimum B6/Cast SNP differences for HIGH_SNP file
RT_COVERAGE_DOWNSTREAM = 100  # nt downstream for SNP detection

# RTBC barcode
ADD_RTBC_BARCODE = True
RTBC_SEQUENCE = "/5Phos/TGACTTGAGGAT"

# ========================================
# Build configuration dictionary
# ========================================
config = FISH_RT_CONFIG.copy()
config.update({
    "gene_list": GENE_LIST,
    "output_directory": str(OUTPUT_DIR),
    "local_gtf_path": GTF_PATH,
    "local_genome_fasta_path": FASTA_PATH,
    "snp_file_path": VCF_PATH,
    "vcf_b6_sample": VCF_B6_SAMPLE,
    "vcf_cast_sample": VCF_CAST_SAMPLE,
    "min_snp_coverage_for_final": MIN_SNP_COVERAGE,
    "rt_coverage_downstream": RT_COVERAGE_DOWNSTREAM,
    "add_rtbc_barcode": ADD_RTBC_BARCODE,
    "rtbc_sequence": RTBC_SEQUENCE,
})

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "gene_sequences").mkdir(exist_ok=True)

print("📋 Configuration:")
print(f"   Genes: {len(GENE_LIST)} ({', '.join(GENE_LIST[:3])}{'...' if len(GENE_LIST) > 3 else ''})")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Min SNP coverage: {MIN_SNP_COVERAGE}")
print(f"   Probes per gene: {PROBES_PER_GENE}")

---

# Step 1: Probe Design

Design focused RT primers using the Oligostan algorithm with quality filtering and SNP analysis.

### 1.1 Initialize Pipeline Components

In [ ]:
# Initialize pipeline components
print("🔧 Initializing pipeline components...")

gene_fetcher = GeneSequenceFetcher(config)
snp_analyzer = SNPCoverageAnalyzer(config) if config["snp_file_path"] else None
output_generator = OutputGenerator(config)

print("✅ Pipeline components initialized")

### 1.2 Fetch Gene Sequences

In [ ]:
# Fetch gene sequences from GTF + FASTA
print(f"📥 Fetching sequences for {len(GENE_LIST)} genes...\n")

all_gene_data = []
failed_genes = []

for gene_name in GENE_LIST:
    try:
        gene_data = gene_fetcher.fetch_gene_sequence(gene_name)
        if gene_data:
            all_gene_data.append(gene_data)
            print(f"✅ {gene_name}: {len(gene_data['sequence'])} bp, strand {gene_data['strand']}")
        else:
            failed_genes.append(gene_name)
            print(f"❌ {gene_name}: Failed to fetch")
    except Exception as e:
        failed_genes.append(gene_name)
        print(f"❌ {gene_name}: Error - {str(e)}")

if failed_genes:
    print(f"\n⚠️ Failed genes: {failed_genes}")

print(f"\n📊 Successfully fetched: {len(all_gene_data)}/{len(GENE_LIST)} genes")

# Save gene sequences for reference
gene_fetcher.save_gene_sequences(all_gene_data, OUTPUT_DIR / "gene_sequences")

### 1.3 Design Probes

In [ ]:
# Design FISH probes using Oligostan algorithm
print("🧬 Designing probes using Oligostan algorithm...\n")

all_probes = []

for gene_data in all_gene_data:
    try:
        probes = design_fish_probes(gene_data, config)
        if probes:
            all_probes.extend(probes)
            print(f"✅ {gene_data['gene_name']}: {len(probes)} probes designed")
        else:
            print(f"⚠️ {gene_data['gene_name']}: No probes generated")
    except Exception as e:
        print(f"❌ {gene_data['gene_name']}: Error - {str(e)}")

print(f"\n📊 Total probes designed: {len(all_probes)}")

### 1.4 Apply Quality Filters

In [ ]:
# Apply stringent filtering (GC + PNAS + Dustmasker)
print("🔍 Applying quality filters...\n")

filtered_probes = []
filter_stats = {
    "total": len(all_probes),
    "gc_pass": 0,
    "pnas_pass": 0,
    "dustmasker_pass": 0,
    "all_filters_pass": 0,
}

for probe in all_probes:
    gc_pass = probe["GCFilter"] == 1
    pnas_pass = probe["PNASFilter"] == 1
    dustmasker_pass = probe.get("MaskedFilter", 1) == 1
    
    if gc_pass:
        filter_stats["gc_pass"] += 1
    if pnas_pass:
        filter_stats["pnas_pass"] += 1
    if dustmasker_pass:
        filter_stats["dustmasker_pass"] += 1
    
    if gc_pass and pnas_pass and dustmasker_pass:
        filtered_probes.append(probe)
        filter_stats["all_filters_pass"] += 1

print(f"📊 Filter Results:")
print(f"   Total probes: {filter_stats['total']}")
print(f"   GC filter pass: {filter_stats['gc_pass']}")
print(f"   PNAS filter pass: {filter_stats['pnas_pass']}")
print(f"   Dustmasker pass: {filter_stats['dustmasker_pass']}")
print(f"   All filters pass: {filter_stats['all_filters_pass']}")
print(f"   Retention rate: {filter_stats['all_filters_pass']/filter_stats['total']*100:.1f}%")

### 1.5 SNP Coverage Analysis

In [ ]:
# Analyze SNP coverage in RT region
if snp_analyzer:
    print(f"🧬 Analyzing SNP coverage ({RT_COVERAGE_DOWNSTREAM} nt downstream)...\n")
    
    filtered_probes = snp_analyzer.analyze_probes(filtered_probes)
    
    # Statistics
    snp_counts = [p.get("SNPs_Covered_Count", 0) for p in filtered_probes]
    avg_snps = sum(snp_counts) / len(snp_counts) if snp_counts else 0
    max_snps = max(snp_counts) if snp_counts else 0
    high_snp_count = sum(1 for c in snp_counts if c >= MIN_SNP_COVERAGE)
    
    print(f"\n📊 SNP Coverage Statistics:")
    print(f"   Average SNPs per probe: {avg_snps:.1f}")
    print(f"   Maximum SNPs per probe: {max_snps}")
    print(f"   Probes with ≥{MIN_SNP_COVERAGE} SNPs: {high_snp_count}")
else:
    print("⚠️ SNP analysis skipped (no VCF file configured)")

### 1.6 Generate Step 1 Outputs

In [ ]:
# Generate output files
print("💾 Generating Step 1 output files...\n")

output_files = output_generator.generate_focused_outputs(filtered_probes, OUTPUT_DIR)

print(f"\n✅ Step 1 Complete!")
print(f"📁 Output files:")
for f in output_files:
    print(f"   {f.name}")

In [ ]:
# Display summary table
df_filtered = pd.read_csv(OUTPUT_DIR / "FISH_RT_probes_FILTERED.csv")

print(f"\n📊 Step 1 Summary:")
print(f"   Total filtered probes: {len(df_filtered)}")
print(f"   Genes: {df_filtered['GeneName'].nunique()}")
print(f"   Avg SNPs covered: {df_filtered['SNPs_Covered_Count'].mean():.1f}")
print(f"   Avg PNAS score: {df_filtered['NbOfPNAS'].mean():.1f}/5")

# Show first few rows
display(df_filtered[['GeneName', 'Seq', 'ProbeSize', 'SNPs_Covered_Count', 'NbOfPNAS', 'dGScore']].head(10))

---

# Step 2: Top Probe Selection

Select the top N probes per gene based on SNP coverage, PNAS score, and thermodynamic score.

In [ ]:
# Load filtered probes
filtered_csv = OUTPUT_DIR / "FISH_RT_probes_FILTERED.csv"
df_filtered = pd.read_csv(filtered_csv)

print(f"📥 Loaded {len(df_filtered)} filtered probes from {df_filtered['GeneName'].nunique()} genes")

In [ ]:
# Select top probes per gene
print(f"🎯 Selecting top {PROBES_PER_GENE} probes per gene...\n")

top_probes = select_top_probes(df_filtered, PROBES_PER_GENE)

# Report selection statistics
print("📊 Selected probes per gene:")
gene_counts = top_probes["GeneName"].value_counts().sort_index()
for gene, count in gene_counts.items():
    avg_snps = top_probes[top_probes["GeneName"] == gene]["SNPs_Covered_Count"].mean()
    avg_pnas = top_probes[top_probes["GeneName"] == gene]["NbOfPNAS"].mean()
    print(f"   {gene}: {count} probes (avg SNPs: {avg_snps:.1f}, avg PNAS: {avg_pnas:.1f}/5)")

print(f"\n✅ Total selected: {len(top_probes)} probes")

In [ ]:
# Save top probes
top_csv = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}.csv"
top_fasta = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}.fasta"

top_probes.to_csv(top_csv, index=False)
num_seqs = generate_fasta(top_probes, top_fasta)

print(f"💾 Saved: {top_csv.name}")
print(f"💾 Saved: {top_fasta.name} ({num_seqs} sequences)")

In [ ]:
# Display top probes
display(top_probes[['GeneName', 'Seq', 'ProbeSize', 'SNPs_Covered_Count', 'NbOfPNAS', 'dGScore']].head(15))

---

# Step 3: BLAST Validation (Manual)

Submit the FASTA sequences to NCBI BLAST to check for off-target binding.

## Instructions:

1. **Copy the FASTA content** from the cell below
2. **Go to NCBI BLAST**: [https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn)
3. **Paste sequences** into the query box
4. **Database**: Select "Nucleotide collection (nr/nt)" or "Mouse genomic + transcript"
5. **Organism**: Mus musculus (taxid:10090)
6. **Run BLAST** and wait for results
7. **Download results** as "Text" format
8. **Save the file** and proceed to Step 4

In [ ]:
# Display FASTA content for copy/paste
print(f"📋 FASTA content for BLAST ({top_fasta.name}):\n")
print("=" * 60)

with open(top_fasta, "r") as f:
    fasta_content = f.read()
    print(fasta_content)

print("=" * 60)
print(f"\n📊 Total sequences: {len(top_probes)}")

---

# Step 4: BLAST Analysis

Analyze BLAST results and filter for probes with unique genomic hits.

In [ ]:
# ========================================
# SPECIFY YOUR BLAST RESULTS FILE PATH
# ========================================

BLAST_RESULTS_FILE = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}-blast.txt"

# Check if file exists
if BLAST_RESULTS_FILE.exists():
    print(f"✅ BLAST file found: {BLAST_RESULTS_FILE.name}")
else:
    print(f"⚠️ BLAST file not found: {BLAST_RESULTS_FILE}")
    print("   Please update BLAST_RESULTS_FILE path and re-run this cell")

In [ ]:
# Parse BLAST results (only run if file exists)
if BLAST_RESULTS_FILE.exists():
    print(f"📖 Reading BLAST results...\n")
    
    with open(BLAST_RESULTS_FILE, "r") as f:
        blast_text = f.read()
    
    blast_df = parse_blast_results(blast_text)
    print(f"✅ Parsed {len(blast_df)} probe results")
else:
    print("⚠️ Skipping BLAST analysis - file not found")
    blast_df = None

In [ ]:
# Merge BLAST results with probe data
if blast_df is not None:
    print("🔗 Merging BLAST results with probe data...\n")
    
    # Load top probes CSV
    csv_df = pd.read_csv(top_csv)
    
    # Create matching keys
    csv_df["ProbeKey"] = csv_df.apply(create_probe_identifier, axis=1)
    blast_df["ProbeKey"] = blast_df.apply(
        lambda row: (
            f"{row['GeneName']}_{row['ProbeSequence']}"
            if pd.notna(row["GeneName"]) and pd.notna(row["ProbeSequence"])
            else None
        ),
        axis=1,
    )
    
    # Merge
    merged_df = csv_df.merge(
        blast_df[["ProbeKey", "ProbeName", "PercentAlignment", "NumberOfHits", "UniqueHitName", "Start", "End"]],
        on="ProbeKey",
        how="left",
    )
    merged_df.drop(columns=["ProbeKey"], inplace=True)
    
    # Filter for unique hits
    unique_df = merged_df[merged_df["NumberOfHits"] == 1].copy()
    
    print(f"📊 BLAST Analysis Summary:")
    print(f"   Total probes: {len(csv_df)}")
    print(f"   Probes with BLAST results: {len(merged_df.dropna(subset=['NumberOfHits']))}")
    print(f"   Probes with unique hits: {len(unique_df)}")
    print(f"   Probes removed (multiple hits): {len(merged_df) - len(unique_df)}")

In [ ]:
# Display hit distribution
if blast_df is not None:
    print("📈 Hit Count Distribution:")
    hit_counts = merged_df["NumberOfHits"].value_counts().sort_index()
    for hits, count in hit_counts.items():
        if pd.notna(hits):
            print(f"   {int(hits)} hit(s): {count} probes")
    
    print("\n🧬 Gene-level Summary:")
    gene_summary = (
        merged_df.groupby("GeneName")
        .agg({"NumberOfHits": ["count", lambda x: (x == 1).sum()]})
        .round(2)
    )
    gene_summary.columns = ["Total", "Unique"]
    
    for gene, row in gene_summary.iterrows():
        total = int(row["Total"])
        unique = int(row["Unique"])
        pct = unique / total * 100 if total > 0 else 0
        print(f"   {gene}: {unique}/{total} unique hits ({pct:.1f}%)")

In [ ]:
# Save BLAST analysis results
if blast_df is not None:
    print("💾 Saving BLAST analysis results...\n")
    
    # Save merged data
    merged_csv = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}_with_BLAST.csv"
    merged_df.to_csv(merged_csv, index=False)
    print(f"   {merged_csv.name}")
    
    # Save unique hits
    unique_csv = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}_UNIQUE_HITS.csv"
    unique_df.to_csv(unique_csv, index=False)
    print(f"   {unique_csv.name}")
    
    # Save unique hits FASTA
    if len(unique_df) > 0:
        unique_fasta = OUTPUT_DIR / f"FISH_RT_probes_TOP{PROBES_PER_GENE}_UNIQUE_HITS.fasta"
        
        records = []
        for idx, row in unique_df.iterrows():
            gene_name = row.get("GeneName", "Unknown")
            sequence = row.get("Seq", "")
            snp_count = row.get("SNPs_Covered_Count", 0)
            pnas_score = row.get("NbOfPNAS", 0)
            
            header = f"{gene_name}_unique_{idx}"
            description = f"Gene:{gene_name} | SNPs:{snp_count} | PNAS:{pnas_score} | BLAST_unique"
            
            record = SeqRecord(Seq(sequence), id=header, description=description)
            records.append(record)
        
        SeqIO.write(records, unique_fasta, "fasta")
        print(f"   {unique_fasta.name} ({len(records)} sequences)")
    
    print(f"\n✅ BLAST analysis complete!")

---

# Final Summary

Display the final results and synthesis-ready probes.

In [ ]:
# Final summary
print("🎉 FISH-RT Probe Design Pipeline Complete!")
print("=" * 50)

print(f"\n📊 Pipeline Summary:")
print(f"   Genes processed: {len(GENE_LIST)}")
print(f"   Total probes designed: {len(all_probes)}")
print(f"   After quality filtering: {len(filtered_probes)}")
print(f"   Top probes selected: {len(top_probes)}")

if blast_df is not None and len(unique_df) > 0:
    print(f"   Unique BLAST hits: {len(unique_df)}")
    print(f"\n✅ Final synthesis-ready probes: {len(unique_df)}")
else:
    print(f"\n⚠️ BLAST analysis not completed - run Step 4 after BLAST validation")

print(f"\n📁 Output directory: {OUTPUT_DIR}")

In [ ]:
# List all output files
print("📄 Output Files:")
for f in sorted(OUTPUT_DIR.glob("*.csv")):
    print(f"   {f.name}")
for f in sorted(OUTPUT_DIR.glob("*.fasta")):
    print(f"   {f.name}")
for f in sorted(OUTPUT_DIR.glob("*.txt")):
    print(f"   {f.name}")

In [ ]:
# Display final unique probes (if available)
if blast_df is not None and len(unique_df) > 0:
    print("🧬 Final Unique Probes (synthesis-ready):")
    display(unique_df[['GeneName', 'Seq', 'RTBC_5Prime_Sequence', 'SNPs_Covered_Count', 'NbOfPNAS', 'NumberOfHits']].head(20))
else:
    print("💡 Complete BLAST validation (Step 3-4) to see final synthesis-ready probes")